# 🔌 Plugins (with semantic functions)

https://docs.google.com/presentation/d/1ansUV9VxvNVfLnB3xerFQypWAKvmv8QLhjuJqeQK2Dg/edit#slide=id.g284f11586a4_0_8

### Initialize the kernel

In [1]:
#r "nuget: Microsoft.SemanticKernel, 1.7.1"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.7.1-alpha"

#!import Settings.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;

var kernel = Settings.InitializeKernel();

Installed Packages Microsoft.SemanticKernel, 1.7.1 Microsoft.SemanticKernel.Plugins.Core, 1.7.1-alpha

### ⌚️ Let's building a plugin consisting of semantic functions.

### Now the kernel contains all plugins ever added, unless we start removing them.
Entire plugins or individual functions can be removed from the kernel anytime.

In [3]:
//var plugin = kernel.ImportPluginFromFunctions()
var plugin = kernel.ImportPluginFromObject() // native
//var plugin = kernel.ImportPluginFromPromptDirectory() semantic
//var plugin = kernel.ImportPluginFromType<>() // native or semantic from native
//var plugin = kernel.ImportPluginFromOpenAIAsync
kernel.CreateFunctionFromMethod()
kernel.CreateFunctionFromPrompt()
kernel.CreatePluginFromFunctions()
kernel.CreatePluginFromObject()
kernel.CreatePluginFromPromptDirectory()
kernel.CreatePluginFromType<>()
kernel.Plugins.Add(plugin);

agent = await CreateAgentBuilder()
    .FromTemplate(definition)
    .WithPlugin(plugin)
    
//ImportPluginFromApiManifestAsync
kernel.ImportPluginFromPromptDirectory(Path.Combine(folder, "SummarizePlugin"));

//agent as plugin

var plugin = KernelPluginFactory.CreateFromFunctions("spelling", "Spelling functions", [function]);
var plugin = KernelPluginFactory.CreateFromObject()
var plugin = KernelPluginFactory.CreateFromType<>()





Error: (1,53): error CS0246: The type or namespace name 'Orchestrator' could not be found (are you missing a using directive or an assembly reference?)
(1,33): error CS1061: 'Kernel' does not contain a definition for 'ImportFunctions' and no accessible extension method 'ImportFunctions' accepting a first argument of type 'Kernel' could be found (are you missing a using directive or an assembly reference?)

Semantic code gets executed in the LLM.
We can ground the AI in knowledge of the world outside LLM to know about today's date just by giving it a plugin that can access native code capabilities.

### Let's prepare the execution settings and the kernel arguments.

In [3]:
var executionSettings = new OpenAIPromptExecutionSettings 
{
     MaxTokens = 2000,
     ResultsPerPrompt = 1,
     Temperature = 0.1
};

var kernelArguments = new KernelArguments(executionSettings)
{
    ["input"] = "Christmas"
};

### 🪴 This simple bit of "grounding" reduces so-called 'hallucination' with LLMs

In this first example, we invoke the kernel with a prompt that asks the AI for information it cannot provide. It has no idea what today might be for you, so it's likely to hallucinate.

In [4]:
var promptTemplate = "How many days until {{$input}}?";

var semanticFunction = kernel.CreateFunctionFromPrompt(promptTemplate);

var response = await kernel.InvokeAsync(semanticFunction, kernelArguments);

display(response.GetValue<string>())

Since today is April 2, 2023, there are 267 days until Christmas on December 25, 2023.

In this second example, the plugin we've added `DateTimePlugin` is used in the templated prompt. The native function GetCurrentDate is invoked to enrich the prompt context with current date.

In [5]:
var promptTemplate = "Considering that today date is {{date.GetCurrentDate}}, how many days until {{$input}}?";

var semanticFunction = kernel.CreateFunctionFromPrompt(promptTemplate);

var response = await kernel.InvokeAsync(semanticFunction, kernelArguments);

display(response.GetValue<string>())

If today is April 20, 2024, we need to calculate the number of days until December 25, 2024.

From April 20 to April 30, there are 10 days (since April has 30 days).
The entire months of May, June, July, August, September, October, and November then follow. Here are the days in each month:

- May: 31 days
- June: 30 days
- July: 31 days
- August: 31 days
- September: 30 days
- October: 31 days
- November: 30 days

Adding these up gives us:

31 (May) + 30 (June) + 31 (July) + 31 (August) + 30 (September) + 31 (October) + 30 (November) = 214 days

Finally, we add the days from December up to the 25th, which is 25 days.

So the total number of days from April 20 to December 25 is:

10 (remaining days in April) + 214 (full months between April and December) + 25 (days in December) = 249 days

Therefore, there are 249 days until Christmas from April 20, 2024.

### Let's check the rendered prompt.

In [11]:
#!import Helpers/PrintHelper.cs

await PrintHelper.PrintRenderedPromptAsync(kernel, promptTemplate, kernelArguments);

Considering that today date is 4/20/2024, how many days until Christmas?


### For a better user experience we can may want to use with streaming.

In [ ]:
await foreach (var token in kernel.InvokePromptStreamingAsync(promptTemplate, kernelArguments))
{
    Console.Write(token);
}